In [2]:
from keras.models import Model 
from keras.layers import Input, BatchNormalization, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Conv1D, UpSampling1D, Conv1DTranspose, Reshape
import np_utils
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
import os
from sklearn.preprocessing import StandardScaler
import os
import datetime
from sklearn.model_selection import train_test_split

2024-11-11 15:57:05.255798: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
poor = pd.read_csv("SimData/bank_reserves_outputs_poor.csv", header=None)
middle = pd.read_csv("SimData/bank_reserves_outputs_middle.csv", header=None)
rich = pd.read_csv("SimData/bank_reserves_outputs_rich.csv", header=None)
sc = StandardScaler()
bank_reserves = []
for i in np.arange(0, poor.shape[0]):
    sample = pd.concat([poor.iloc[i], middle.iloc[i]], axis=0).T
    sample = pd.concat([sample, rich.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    bank_reserves.append(sample_std)

In [7]:
def DCAE_reduction(df):
    
    df_flat = [sample.flatten() for sample in df]
    df_flat = np.asarray(df_flat)
    dim = df_flat.shape[1]
    train, test = train_test_split(df_flat, test_size=0.20, random_state=42)
    train = train.reshape(train.shape[0], dim, 1)
    test = test.reshape(test.shape[0], dim, 1)
    
    input = Input(shape=(dim,1))

    encoded = Conv1D(filters=32, kernel_size=10, activation='relu', padding='same')(input)
    encoded = BatchNormalization()(encoded)
    encoded = Conv1D(filters=64, kernel_size=10, activation='relu', padding='same')(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Conv1D(filters=128, kernel_size=10, activation='relu', padding='same')(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(encoded)
    encoded = BatchNormalization()(encoded)

    encoded = Flatten()(encoded)

    encoded = Dense(dim, activation='relu')(encoded)
    decoded = Dense(5, activation='relu')(encoded)
    decoded = Dense(dim, activation='relu')(decoded)

    decoded = Reshape((dim, 1))(decoded)

    decoded = Conv1DTranspose(filters=128, kernel_size=3, activation='relu', padding='same')(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Conv1DTranspose(filters=128, kernel_size=10, activation='relu', padding='same')(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Conv1DTranspose(filters=64, kernel_size=10, activation='relu', padding='same')(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Conv1DTranspose(filters=32, kernel_size=10, activation='relu', padding='same')(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Conv1DTranspose(filters=1, kernel_size=10, activation=None, padding='same')(decoded)

    autoencoder = Model(input, decoded)
    autoencoder.summary()
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    autoencoder.fit(train, train,
     epochs=20,
     batch_size=64,
     shuffle=True,
     validation_data=(test, test),
     verbose=1)
    encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=11).output)
    encoded_ts = encoder.predict(df_flat)
    return encoded_ts

In [8]:
br_dcae = DCAE_reduction(bank_reserves)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 303, 1)]          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 303, 32)           352       
                                                                 
 batch_normalization_8 (Batc  (None, 303, 32)          128       
 hNormalization)                                                 
                                                                 
 conv1d_5 (Conv1D)           (None, 303, 64)           20544     
                                                                 
 batch_normalization_9 (Batc  (None, 303, 64)          256       
 hNormalization)                                                 
                                                                 
 conv1d_6 (Conv1D)           (None, 303, 128)          8204

In [9]:
np.savetxt("extracted_features/bank_reserves_DCAE.csv", encoded_ts, delimiter=",")

NameError: name 'encoded_ts' is not defined

In [11]:
ecv_active = pd.read_csv("SimData/epsteinCV_outputs_active.csv", header=None)
ecv_jailed = pd.read_csv("SimData/epsteinCV_outputs_jailed.csv", header=None)
ecv_quiet = pd.read_csv("SimData/epsteinCV_outputs_quiet.csv", header=None)
sc = StandardScaler()
ecv = []
for i in np.arange(0, ecv_active.shape[0]):
    sample = pd.concat([ecv_active.iloc[i], ecv_jailed.iloc[i]], axis=0).T
    sample = pd.concat([sample, ecv_quiet.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    ecv.append(sample_std)

In [ ]:
ecv_dcae = DCAE_reduction(ecv)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 756, 1)]          0         
                                                                 
 conv1d_8 (Conv1D)           (None, 756, 32)           352       
                                                                 
 batch_normalization_16 (Bat  (None, 756, 32)          128       
 chNormalization)                                                
                                                                 
 conv1d_9 (Conv1D)           (None, 756, 64)           20544     
                                                                 
 batch_normalization_17 (Bat  (None, 756, 64)          256       
 chNormalization)                                                
                                                                 
 conv1d_10 (Conv1D)          (None, 756, 128)          8204

In [ ]:
np.savetxt("extracted_features/epstein_DCAE.csv", ecv_dcae, delimiter=",")

In [10]:
np.savetxt("extracted_features/bank_reserves_DCAE.csv", br_dcae, delimiter=",")

In [ ]:
def import_ff_data(filename):
    expected_columns=155
    data = []
    with open(filename, 'r') as file:
        for line in file:
            row = line.strip().split(',')
            if len(row) < expected_columns:
                row += [np.nan] * (expected_columns - len(row))
            data.append(row)
    df = pd.DataFrame(data)
    def fill_last_valid(row):
        for i in range(1, len(row)):
            if pd.isna(row[i]):
                row[i] = row[i - 1]  
        return row
    df_filled = df.apply(fill_last_valid, axis=1)
    return df_filled

In [ ]:
ff_onfire = import_ff_data("SimData/forest_fire_outputs_onfire.csv")
print("check 1")
ff_fine = import_ff_data("SimData/forest_fire_outputs_fine.csv")
print("check 2")
ff_burned = import_ff_data("SimData/forest_fire_outputs_burned.csv")
sc = StandardScaler()
ff = []
for i in np.arange(0, ff_onfire.shape[0]):
    sample = pd.concat([ff_onfire.iloc[i], ff_fine.iloc[i]], axis=0).T
    sample = pd.concat([sample, ff_burned.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    ff.append(sample_std)

In [ ]:
ff_dcae = DCAE_reduction(ff)

In [ ]:
np.savetxt("extracted_features/ff_DCAE.csv", ff_dcae, delimiter=",")